In [ ]:
# Ocultar/Mostrar código automaticamente
from IPython.display import HTML
HTML('''
<script>
  code_show=true; 
  function code_toggle() {
    if (code_show){
      $('div.input').hide();
    } else {
      $('div.input').show();
    }
    code_show = !code_show
  } 
  $( document ).ready(code_toggle);
</script>
<a href="javascript:code_toggle()">Clique aqui para mostrar/ocultar código</a>.
''')

In [ ]:
# Conversor BPA -> Plano de Ação (com Upload e Download)
!pip install openpyxl pandas requests ipywidgets

import pandas as pd
import openpyxl
from openpyxl.drawing.image import Image
from openpyxl.styles import Alignment
from datetime import datetime
from google.colab import files
import requests
import ipywidgets as widgets
from IPython.display import display, clear_output

# ======= CONFIGURAÇÕES =======
BASE_URL = "https://github.com/fabiaalves/conversor-planner-bahia/raw/main/"
URL_MAPEAMENTO = BASE_URL + "Mapeamento_Colunas.xlsx"
URL_MODELO = BASE_URL + "PLANO_DE_ACAO.xlsx"
URL_IMAGEM = BASE_URL + "nova_imagem.png"

# ======= BAIXAR ARQUIVOS FIXOS DO GITHUB =======
def baixar_arquivo(url, nome_arquivo):
    r = requests.get(url)
    if r.status_code != 200:
        raise Exception(f"Erro ao baixar {url}. Verifique se o arquivo existe no repositório.")
    with open(nome_arquivo, 'wb') as f:
        f.write(r.content)

baixar_arquivo(URL_MAPEAMENTO, "mapeamento.xlsx")
baixar_arquivo(URL_MODELO, "modelo.xlsx")
baixar_arquivo(URL_IMAGEM, "imagem_topo.png")

# ======= INTERFACE =======
uploader = widgets.FileUpload(accept='.xlsx', multiple=False)
botao_converter = widgets.Button(description='Converter para Plano de Ação')
saida = widgets.Output()

def converter_planilha(uploaded_file):
    df_entrada = pd.read_excel(pd.io.common.BytesIO(uploaded_file), engine="openpyxl")
    df_mapeamento = pd.read_excel("mapeamento.xlsx", engine="openpyxl")

    wb = openpyxl.load_workbook("modelo.xlsx")
    ws = wb.active

    # Substituir imagem do topo
    ws._images = []  # limpa imagens antigas
    nova_img = Image("imagem_topo.png")
    nova_img.anchor = 'A1'
    ws.add_image(nova_img)

    # Ler cabeçalhos do modelo
    headers_modelo = [cell.value for cell in ws[1]]

    # Preencher dados conforme mapeamento
    for _, row in df_mapeamento.iterrows():
        origem = row.get('Coluna do Planner (origem)')
        destino = row.get('Coluna na nova planilha (destino)')
        regra = str(row.get('Transformação / Regra especial', '')).strip().lower()
        formato = row.get('Formato esperado', '')

        if pd.isna(destino) or destino not in headers_modelo:
            continue

        col_idx = headers_modelo.index(destino) + 1

        for i in range(len(df_entrada)):
            valor = ""
            if regra == "valor fixo":
                valor = formato
            elif regra == "data (dd/mm/yyyy)" and origem in df_entrada.columns:
                valor = pd.to_datetime(df_entrada.loc[i, origem], errors='coerce').strftime('%d/%m/%Y') if pd.notna(df_entrada.loc[i, origem]) else ""
            elif origem in df_entrada.columns:
                valor = df_entrada.loc[i, origem]
            ws.cell(row=i+2, column=col_idx, value=valor)

    # Aplicar quebra de texto e ajuste de largura
    for col in ws.columns:
        max_length = 0
        column = col[0].column_letter
        for cell in col:
            cell.alignment = Alignment(wrap_text=True, vertical='top')
            if cell.value:
                max_length = max(max_length, len(str(cell.value)))
        ws.column_dimensions[column].width = min(max_length + 2, 50)
    for row in ws.iter_rows(min_row=2):
        ws.row_dimensions[row[0].row].height = 30

    # Salvar arquivo final
    hoje = datetime.now().strftime('%d-%m-%Y')
    nome_saida = f"PLANO_DE_ACAO_FINAL_{hoje}.xlsx"
    wb.save(nome_saida)
    return nome_saida

def on_button_click(b):
    with saida:
        clear_output()
        if len(uploader.value) == 0:
            print("⚠️ Nenhum arquivo enviado.")
            return
        for nome_arquivo, file_info in uploader.value.items():
            content = file_info['content']
            nome_saida = converter_planilha(content)
            files.download(nome_saida)
            print(f"✔️ Conversão concluída! Arquivo gerado: {nome_saida}")

botao_converter.on_click(on_button_click)

display(uploader, botao_converter, saida)